In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 10.7 MB/s eta 0:00:00


In [2]:
import sys
!{sys.executable} -m pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 1.7 MB/s eta 0:00:00


In [3]:
import sys
!{sys.executable} -m pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.1 MB/s eta 0:00:00


In [4]:
import sys
!{sys.executable} -m pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=714046e8ec4aee9442a459aa89840202ca54cc7514e2cb0a73e373a407b43318
  Stored in directory: /home/idies/.cache/pip/wheels/20/7c/09/4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.29
    Uninstalling GitPython-3.1.29:
      Successfully uninstalled GitPython-3.1.29


In [5]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [6]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='XGBoost Classifier',job_type='analysis',reinit=True)

wandb: Currently logged in as: sakhawat. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /home/idies/workspace/Temporary/s.hossain18/scratch/jobs/20230809/20230809001913-245606/wandb/run-20230809_075119-55m47kly
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run XGBoost Classifier
wandb: ⭐️ View project at https://wandb.ai/sakhawat/exoplanetml
wandb: 🚀 View run at https://wandb.ai/sakhawat/exoplanetml/runs/55m47kly


In [7]:
import numpy as np

In [8]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [9]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [10]:
from skopt import BayesSearchCV
import xgboost as xgb

In [11]:
# Create an instance of the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(random_state=43,n_jobs=-1)

In [12]:
# Define the parameter grid for XGBoost
params = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.2),
    'n_estimators': (400, 500),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1)
}

In [13]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [14]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(xgb_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [15]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=41),
              estimator=XGBClassifier(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric=None, feature_types=None,
                                      gamma=None, gpu_id=None, grow_policy=None,
                                      impo...
                                      max_leaves=None, min_child_weight=None,
                                      missing=nan, monotone_constraints=None,
                                      n_estimators=100, n_jobs=-1,
                                      num_parallel_tree=None, predictor=None,
                                      random_state=43, ...),
              n_iter=20, n_jobs=-1,
              search_spaces={'colsample_bytree': (0.5, 1.0),
                             'learning_rate': (0.01, 0.2), 'max_depth': (3, 10),
                             'n_estimators': (400, 500), 'reg_alpha': (0, 1),
                             'reg_lambda': (0, 1), 'subsample': (0.5, 1.0)})

In [16]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

Train score: 0.8100194865712108
Test score: 0.8249630723781388
Best hyperparameters: OrderedDict([('colsample_bytree', 0.8840923286188427), ('learning_rate', 0.02136758382220562), ('max_depth', 10), ('n_estimators', 426), ('reg_alpha', 0), ('reg_lambda', 1), ('subsample', 0.8834671223572841)])


In [17]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [18]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:  Test score ▁
wandb: Train score ▁
wandb: 
wandb: Run summary:
wandb:  Test score 0.82496
wandb: Train score 0.81002
wandb: 
wandb: 🚀 View run XGBoost Classifier at: https://wandb.ai/sakhawat/exoplanetml/runs/55m47kly
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230809_075119-55m47kly/logs
